In [ ]:
# Practicumopdrachten Week 2.2 (eerste kans)

## Dependencies importeren

Importeer in het codeblok hieronder de packages die worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
N.B.: de 2 reeds geschreven coderegels zorgen ervoor dat eventuele warnings, die code-outputs lelijker maken, uitgezet worden.

In [4]:

import warnings
import numpy as np
import sqlite3
import pandas as pd
warnings.simplefilter('ignore')

ModuleNotFoundError: No module named 'numpy'

Zet het bestand go_sales.sqlite in hetzelfde mapje als dit notebook.

## Databasetabel inlezen

Creëer een databaseconnectie met het bestand go_sales.sqlite.

In [3]:
con = sqlite3.connect("Assets/Week 2/go_sales.sqlite")

Gebruik de onderstaande sql_query om te achterhalen welke databasetabellen in go_sales zitten.

In [4]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
#Vul dit codeblok verder in
def discoverTables() :
    try :
        con = sqlite3.connect("Assets/Week 2/go_sales.sqlite")
        cursor = con.cursor() 
        print("Connected to sqlite.")
        cursor.execute(sql_query)
        records = cursor.fetchall()
        print("Total rows are:  ", len(records))
        for row in records : 
            print(f"    {row}")
    except sqlite3.Error as error:
        print("Failed to read data from sqlite table", error)
    finally:
        if con:
            con.close()
            print("The SQLite connection is now closed.")

discoverTables()

Gebruik de gecreëerde databaseconnectie om de resultaten van de volgende query in een DataFrame te zetten:<br>
*SELECT * FROM sales_staff* 

In [ ]:
sql_query = "SELECT * FROM sales_staff;"

def dataframe() : 
    try : 
        con = sqlite3.connect("Assets/Week 2/go_sales.sqlite")
        cursor = con.cursor() 
        print("Connected to sqlite.")
        result = pd.read_sql_query(sql_query,con)
        print(result)
    except sqlite3.Error as error: 
        print("Failed to read data from sqlite table", error)
    finally : 
        if con:
            con.close()
            print("The SQLite connection is now closed.")

dataframe()

## DataFrames uitsplitsen en opbouwen met Series

De volgende 5 kolommen hebben betrekking op de contactdetails van elke medewerker in dit DataFrame:
- SALES_STAFF_CODE
- WORK_PHONE
- EXTENSION
- FAX
- EMAIL

Maak van elk van deze 5 kolommen een serie.

In [ ]:
sql_query = "SELECT * FROM sales_staff;"
global sales_staff_column 
global work_phone_column
global extension_column
global fax_column
global email_column


def go_sales_series() : 
   try : 
        con = sqlite3.connect("Assets/Week 2/go_sales.sqlite")
        cursor = con.cursor() 
        print("Connected to sqlite.")
        result =  pd.read_sql_query(sql_query,con)

        global sales_staff_column 
        sales_staff_column = result["SALES_STAFF_CODE"]

        global work_phone_column
        work_phone_column = result["WORK_PHONE"]

        global extension_column
        extension_column = result["EXTENSION"]

        global fax_column
        fax_column = result["FAX"]

        global email_column
        email_column = result["EMAIL"]

        db_series = (sales_staff_column,work_phone_column,extension_column,fax_column,email_column)
        #print(db_series)

        #print(result)
        # print(sales_staff_column)
   except sqlite3.Error as error: 
       print("Failed to read data from sqlite table", error)
   finally : 
        if con:
            con.close()
            print("The SQLite connection is now closed.")

go_sales_series()

#print(sales_staff_column)
#print(work_phone_column)
#print(extension_column)
#print(fax_column)
#print(email_column)

Zet allevijf gecreëerde series als kolommen naast elkaar in een DataFrame (*contact_details*). Gebruik pd.concat(...)<br>
Hulpvraag: welke waarde geef je aan de axis-parameter?

In [ ]:
# print(sales_staff_column)
# print(work_phone_column)
# print(extension_column)
# print(fax_column)
# print(email_column)



contact_details = pd.concat([sales_staff_column,work_phone_column,extension_column,fax_column,email_column], axis=1)
print(contact_details)

## Series en DataFrames maken vanuit lists en dictionaries

Met .head(*getal*) kan je de bovenste *getal* rijen van een tabel selecteren.<br>
Selecteer op deze manier de bovenste 5 rijen van *contact_details*.<br>
Sla dit resultaat op in een nieuw DataFrame.

In [ ]:
# print(contact_details.head(5))
top5_contactdetails = pd.DataFrame(contact_details.head(5))
print(top5_contactdetails)

Aan deze 10 rijen met contactdetails willen we 3 kolommen toevoegen: 'FIRST_LANGUAGE', 'SECOND_LANGUAGE' & 'THIRD_LANGUAGE'.<br>
Iedereens 'First Language' is Engels, afgekort 'EN'. Maak een lijst waarin 5x 'EN' staat.<br>
Converteer deze lijst vervolgens naar een serie en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [ ]:
language_column = pd.Series(["EN","EN","EN","EN","EN"], name="FIRST_LANGUAGE")
truncated_Contact = pd.concat([top5_contactdetails,language_column], axis=1)
print(truncated_Contact)

Maak nu de tweede kolom ('SECOND_LANGUAGE'). Maak daarvoor een dictionary, met daarin...
- Als keys: alle indexen uit het resultaat van het vorige codeblok.
- Als values: bij de eerste 3 elementen 'FR' (Frankrijk), bij de laatste 2 'DE' (Duitsland).

Maak vervolgens ook hier weer een serie van en voeg ook deze weer horizontaal samen met het rsultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [2]:
second_language_column = pd.Series({0:"FR", 1:"FR",2:"FR",3:"DE",4:"DE"}, name="SECOND_LANGUAGE")
truncated_Contact = pd.concat([truncated_Contact, second_language_column], axis=1)
print(truncated_Contact)

NameError: name 'pd' is not defined

Maak ten slotte de derde kolom ('THIRD_LANGUAGE') door een dictionary te maken met daarin...
- Als key: de naam van de nieuwe kolom. Zie je het verschil qua keys met de vorige opdracht?
- Als waarde: een lijst met daarin 'NL', 'NL', 'JPN', 'JPN', 'KOR'.

Converteer deze dictionary nu naar een DataFrame en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Waarom hoef je hierna de nieuw ontstane kolom geen passende naam meer te geven?

In [1]:
third_language_column = pd.Series({"THIRD_LANGUAGE":["NL","NL","JPN","JPN","KOR"]})
df_third_lang = pd.DataFrame(third_language_column)
# print(df_third_lang)
truncated_Contact = pd.concat([truncated_Contact,df_third_lang], axis=1)
print(truncated_Contact)

NameError: name 'pd' is not defined

## Data toevoegen

### Rijen

Gebruik de originele databasetabel *sales_staff*.<br>
Voeg een extra rij toe met eigen invulling. Zorg ervoor dat de index netjes doorloopt.<br>
Hulpvraag: welke waarde geef je nu aan axis?

### Kolommen

Voeg een kolom *FULL_NAME* toe die de waarden van *FIRST_NAME* en *LAST_NAME* achter elkaar zet, gescheiden door een spatie.

## Data wijzigen

### Datatypen

Door het attribuut .dtypes van een DataFrame op te vragen krijg je een serie die per kolom het datatype weergeeft. Doe dit bij de originele databasetabel *sales_staff*

Hier valt op dat elke kolom het datatype 'object' heeft: Python leest dus alles als string. Wiskundige operaties zijn hierdoor niet mogelijk.<br>
We kunnen proberen om kolommen met getallen, bijvoorbeeld de 'extension', te converteren naar een int. Zie onderstaande code.

In [3]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(int)
sales_staff.dtypes

NameError: name 'sales_staff' is not defined

Dit lukt echter niet, omdat er in de kolom 'EXTENSION' lege waarden zitten die niet geconverteerd kunnen worden naar een int.<br>
Wél kan je deze naar een float converteren, zie onderstaande code:

In [ ]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(float)
sales_staff.dtypes

Als we in de rij van 'EXTENSION' kijken zien we dat de conversie van het datatype nu is gelukt.<br>
Dit is <b>randvoorwaardelijk</b> voor het uitvoeren van wiskundige operaties.<br>

### Rijen

Zorg er nu voor dat bij alle extensions 1 wordt opgeteld.

Elke 'Branch Manager' wordt nu 'General Manager'. Schrijf code zodat deze wijziging doorgevoerd wordt in het DataFrame.

### Kolommen

Verander de kolomnaam 'POSITION_EN' naar 'POSITION'.

## Data verwijderen

### Rijen

De medewerkers op indexen 99, 100 en 101 hebben helaas ontslag genomen.<br>
Verwijder de bijbehorende rijen uit het DataFrame. Gebruik slechts één keer de .drop()-methode.

### Kolommen

Faxen zijn inmiddels ouderwets: niemand gebruikt zijn/haar faxnummer nog.<br>
Verwijder de bijbehorende kolom uit het DataFrame.